In [ ]:
pip install transformers

     |████████████████████████████████| 2.1MB 10.3MB/s 
     |████████████████████████████████| 901kB 50.1MB/s 
     |████████████████████████████████| 3.3MB 55.9MB/s 


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  if test is not None:
    validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  else:
    validation_InputExamples = None
  
  return train_InputExamples, validation_InputExamples

  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2, hidden_dropout_prob=0.2)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/training.1600000.processed.noemoticon_simple_proc.csv',encoding='latin-1', header=None)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8
0,NaN,0,1,2,3,4,5,text_proc_simple,target
1,0.0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww bummer shoulda david carr day,0
2,1.0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset update facebook texting cry result schoo...,0
3,2.0,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,dive time ball manage save rest bound,0
4,3.0,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,body feel itchy fire,0


In [ ]:
df = df.dropna(axis='rows')

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8
1,0.0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww bummer shoulda david carr day,0
2,1.0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset update facebook texting cry result schoo...,0
3,2.0,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,dive time ball manage save rest bound,0
4,3.0,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,body feel itchy fire,0
5,4.0,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",behave mad,0


In [ ]:
X = df[7][1:]
y = df[8][1:].apply(lambda t: int(t))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=796953, 
                                                        shuffle=True, stratify=y)
df_train = pd.DataFrame({'txt': X_train, 'target':y_train})
df_test = pd.DataFrame({'txt': X_test, 'target':y_test})
df_train['target'] = df_train['target'].apply(lambda t: 0 if t == 0 else 1)
df_test['target'] = df_test['target'].apply(lambda t: 0 if t == 0 else 1)
df_train = df_train.iloc[:100000]
df_test = df_test.iloc[:20000]

DATA_COLUMN = "txt"
LABEL_COLUMN = "target"

train_InputExamples, validation_InputExamples = convert_data_to_examples(df_train, df_test, DATA_COLUMN, LABEL_COLUMN)
train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
#V2
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=3, validation_data=validation_data)

Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=3, validation_data=validation_data)

Epoch 1/3
   6250/Unknown - 2967s 472ms/step - loss: 0.6944 - accuracy: 0.5008WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
6250/6250 [==============================] - 3071s 488ms/step - loss: 0.6944 - accuracy: 0.5008 - val_loss: 0.6931 - val_accuracy: 0.4961
Epoch 2/3
6250/6250 [==============================] - 3052s 488ms/step - loss: 0.6931 - accuracy: 0.5057 - val_loss: 0.6931 - val_accuracy: 0.4961
Epoch 3/3
6250/6250 [==============================] - 3052s 488ms/step - loss: 0.6931 - accuracy: 0.5005 - val_loss: 0.6931 - val_accuracy: 0.4961


In [ ]:
twitter_labelled_dta = pd.read_json('/content/drive/MyDrive/Colab Notebooks/simple_proc_twitter_labelled.json')
twitter_labelled_dta_1 = twitter_labelled_dta[twitter_labelled_dta['sentiment'] != 0]
twitter_labelled_dta_1['target'] = twitter_labelled_dta_1['sentiment'].apply(lambda t: 0 if t == -1 else 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
twitter_train_InputExamples, twitter_validation_InputExamples = convert_data_to_examples(twitter_labelled_dta_1, None, 'text_simple_proc', 'target')
twitter_train_data = convert_examples_to_tf_dataset(list(twitter_train_InputExamples), tokenizer)
twitter_train_data = twitter_train_data.batch(32)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
model.evaluate(twitter_train_data)

23/23 [==============================] - 4s 163ms/step - loss: nan - accuracy: 0.6288


[nan, 0.628808856010437]

In [ ]:
#NEW 
model.evaluate(twitter_train_data)

23/23 [==============================] - 4s 163ms/step - loss: 1.5047 - accuracy: 0.7452


[1.5046945810317993, 0.7451523542404175]

In [ ]:
model.evaluate(validation_data)

625/625 [==============================] - 103s 165ms/step - loss: 0.6931 - accuracy: 0.4961


[0.6931458711624146, 0.4961499869823456]

In [ ]:
#NEW
model.evaluate(validation_data)

625/625 [==============================] - 104s 166ms/step - loss: 2.0520 - accuracy: 0.7200


[2.0520107746124268, 0.7200499773025513]

In [ ]:
df_test_1 = df_test.iloc[20001:90001]

DATA_COLUMN = "txt"
LABEL_COLUMN = "target"

train_InputExamples_1, validation_InputExamples_1 = convert_data_to_examples(df_test_1, None, DATA_COLUMN, LABEL_COLUMN)
validation_data_1 = convert_examples_to_tf_dataset(list(train_InputExamples_1), tokenizer)
validation_data_1 = validation_data_1.batch(32)


In [ ]:
model.evaluate(validation_data_1)

In [ ]:
reddit_labelled_dta = pd.read_json('/content/drive/MyDrive/Colab Notebooks/simple_proc_reddit_labelled.json')
reddit_labelled_dta_1 = reddit_labelled_dta[reddit_labelled_dta['sentiment'] != 0]

reddit_labelled_dta_1['target'] = reddit_labelled_dta_1['sentiment'].apply(lambda t: 0 if t == -1 else 1)

reddit_train_InputExamples, reddit_validation_InputExamples = convert_data_to_examples(reddit_labelled_dta_1, None, 'text_simple_proc', 'target')
reddit_train_data = convert_examples_to_tf_dataset(list(reddit_train_InputExamples), tokenizer)
reddit_train_data = reddit_train_data.batch(32)

model.evaluate(reddit_train_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


30/30 [==============================] - 5s 166ms/step - loss: 1.5829 - accuracy: 0.6813


[1.582934856414795, 0.6812956929206848]

In [ ]:
insta_labelled_dta = pd.read_json('/content/drive/MyDrive/Colab Notebooks/simple_proc_instagram_labelled.json')
insta_labelled_dta_1 = insta_labelled_dta[insta_labelled_dta['sentiment'] != 0]

insta_labelled_dta_1['target'] = insta_labelled_dta_1['sentiment'].apply(lambda t: 0 if t == -1 else 1)

insta_train_InputExamples, insta_validation_InputExamples = convert_data_to_examples(insta_labelled_dta_1, None, 'text_simple_proc', 'target')
insta_train_data = convert_examples_to_tf_dataset(list(insta_train_InputExamples), tokenizer)
insta_train_data = insta_train_data.batch(32)

model.evaluate(insta_train_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


26/26 [==============================] - 4s 162ms/step - loss: 1.4124 - accuracy: 0.7259


[1.4124138355255127, 0.7259259223937988]

In [ ]:
tiktok_labelled_dta = pd.read_json('/content/drive/MyDrive/Colab Notebooks/simple_proc_tiktok_labelled.json')
tiktok_labelled_dta_1 = tiktok_labelled_dta[tiktok_labelled_dta['sentiment'] != 0]
tiktok_labelled_dta_1['target'] = tiktok_labelled_dta_1['sentiment'].apply(lambda t: 0 if t == -1 else 1)
tiktok_train_InputExamples, tiktok_validation_InputExamples = convert_data_to_examples(tiktok_labelled_dta_1, None, 'text_simple_proc', 'target')
tiktok_train_data = convert_examples_to_tf_dataset(list(tiktok_train_InputExamples), tokenizer)
tiktok_train_data = tiktok_train_data.batch(32)

model.evaluate(tiktok_train_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


21/21 [==============================] - 3s 159ms/step - loss: 1.0371 - accuracy: 0.7960


[1.0371496677398682, 0.795950174331665]